In [12]:
import pandas as pd
import numpy as np  
import seaborn as sns
import matplotlib.pyplot as pyplot
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
import string
from wordcloud import STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier



In [ ]:
df  = pd.read_pickle("df_balanced.pkl")

In [14]:
df.head()

,id,keyword,location,text,target,presence_location,presence_keyword,char_count,word_count,stopword_count,...,keyword_tokens,location_tokens,text_tokens,text_stem,text_lemma,keyword_stem,keyword_lemma,location_stem,location_lemma,text_combined
6818,9765,trapped,10 Steps Ahead. Cloud 9,Bomb head? Explosive decisions dat produced mo...,1,1,1,134,21,6,...,[trapped],"[steps, ahead, cloud]","[bomb, head, explosive, decisions, dat, produc...",bomb head explos decis dat produc dead childre...,bomb head explosive decision dat produced dead...,trap,trapped,step ahead cloud,step ahead cloud,bomb head explosive decision dat produced dead...
4842,6896,mass%20murder,"Huntsville, AL",Okay not sure the word 'mass murder' applies d...,1,1,1,121,19,9,...,[massmurder],"[huntsville, al]","[okay, sure, word, mass, murder, applies, war,...",okay sure word mass murder appli war horrend n...,okay sure word mass murder applies war horrend...,massmurd,massmurder,huntsvill al,huntsville al,okay sure word mass murder applies war horrend...
5848,8356,ruin,Garrett,like why on earth would you want anybody to be...,0,1,1,101,17,6,...,[ruin],[garrett],"[like, earth, would, want, anybody, unhappy, d...",like earth would want anybodi unhappi dont pur...,like earth would want anybody unhappy dont pur...,ruin,ruin,garrett,garrett,like earth would want anybody unhappy dont pur...
1851,2661,crush,"Cleveland, Ohio",My woman crush wedneday goes to the beautiful ...,0,1,1,108,12,3,...,[crush],"[cleveland, ohio]","[woman, crush, wedneday, goes, beautiful]",woman crush wedneday goe beauti,woman crush wedneday go beautiful,crush,crush,cleveland ohio,cleveland ohio,woman crush wedneday go beautiful crush clevel...
5729,8176,rescuers,Washington,#News: 'Many deaths' in shipwreck: Rescuers ar...,1,1,1,136,18,6,...,[rescuers],[washington],"[many, deaths, shipwreck, rescuers, trying, sa...",mani death shipwreck rescuer tri save hundr mi...,many death shipwreck rescuer trying save hundr...,rescuer,rescuer,washington,washington,many death shipwreck rescuer trying save hundr...


In [ ]:
df["has_question"] = df["text"].str.contains(r"\?").astype(int)
df["exclamation_count"] = df["text"].str.count(r"!").astype(int)

In [16]:
# Par exemple pour lemmatized
df["combined_lemma"] = (
    df["text_lemma"] + " " +
    df["keyword_lemma"].fillna("") + " " +
    df["location_lemma"].fillna("")
)

# Pour stemmed
df["combined_stemmed"] = (
    df["text_stem"] + " " +
    df["keyword_stem"].fillna("") + " " +
    df["location_stem"].fillna("")
)



In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4392 entries, 6818 to 2019
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4392 non-null   int64  
 1   keyword            4392 non-null   object 
 2   location           4392 non-null   object 
 3   text               4392 non-null   object 
 4   target             4392 non-null   int64  
 5   presence_location  4392 non-null   int64  
 6   presence_keyword   4392 non-null   int64  
 7   char_count         4392 non-null   int64  
 8   word_count         4392 non-null   int64  
 9   stopword_count     4392 non-null   int64  
 10  unique_word_count  4392 non-null   int64  
 11  stop_word_count    4392 non-null   int64  
 12  url_count          4392 non-null   int64  
 13  mean_word_length   4392 non-null   float64
 14  punctuation_count  4392 non-null   int64  
 15  hashtag_count      4392 non-null   int64  
 16  mention_count      4392 no

In [18]:
df.to_csv("df_balanced.csv", index=False)


In [19]:
df.head()

,id,keyword,location,text,target,presence_location,presence_keyword,char_count,word_count,stopword_count,...,text_lemma,keyword_stem,keyword_lemma,location_stem,location_lemma,text_combined,has_question,exclamation_count,combined_lemma,combined_stemmed
6818,9765,trapped,10 Steps Ahead. Cloud 9,Bomb head? Explosive decisions dat produced mo...,1,1,1,134,21,6,...,bomb head explosive decision dat produced dead...,trap,trapped,step ahead cloud,step ahead cloud,bomb head explosive decision dat produced dead...,1,0,bomb head explosive decision dat produced dead...,bomb head explos decis dat produc dead childre...
4842,6896,mass%20murder,"Huntsville, AL",Okay not sure the word 'mass murder' applies d...,1,1,1,121,19,9,...,okay sure word mass murder applies war horrend...,massmurd,massmurder,huntsvill al,huntsville al,okay sure word mass murder applies war horrend...,0,0,okay sure word mass murder applies war horrend...,okay sure word mass murder appli war horrend n...
5848,8356,ruin,Garrett,like why on earth would you want anybody to be...,0,1,1,101,17,6,...,like earth would want anybody unhappy dont pur...,ruin,ruin,garrett,garrett,like earth would want anybody unhappy dont pur...,0,0,like earth would want anybody unhappy dont pur...,like earth would want anybodi unhappi dont pur...
1851,2661,crush,"Cleveland, Ohio",My woman crush wedneday goes to the beautiful ...,0,1,1,108,12,3,...,woman crush wedneday go beautiful,crush,crush,cleveland ohio,cleveland ohio,woman crush wedneday go beautiful crush clevel...,0,0,woman crush wedneday go beautiful crush clevel...,woman crush wedneday goe beauti crush clevelan...
5729,8176,rescuers,Washington,#News: 'Many deaths' in shipwreck: Rescuers ar...,1,1,1,136,18,6,...,many death shipwreck rescuer trying save hundr...,rescuer,rescuer,washington,washington,many death shipwreck rescuer trying save hundr...,0,0,many death shipwreck rescuer trying save hundr...,mani death shipwreck rescuer tri save hundr mi...


In [20]:
TEXT_COLUMNS = {
    "clean_text": "clean_text", 
    "lemma": "text_lemma",                          
    "combined_lemma": "combined_lemma",
    "stemmed": "text_stem",                       
    "combined_stemmed": "combined_stemmed"
}



# Méta-features numériques à tester
META_FEATURES = ["word_count", "stopword_count", "url_count", "char_count", "punctuation_count", "hashtag_count", "mention_count" , "has_question", "exclamation_count", "presence_keyword", "presence_location"]

# Modèles à tester
MODELS = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(),
    "SVM": SVC(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
import pandas as pd

# Dictionnaires
TEXT_COLUMNS = {
    "clean_text": "clean_text", 
    "lemma": "text_lemma",                          
    "combined_lemma": "combined_lemma",
    "stemmed": "text_stem",                       
    "combined_stemmed": "combined_stemmed"
}

META_FEATURES = [
    "word_count", "stopword_count", "url_count", "char_count",
    "punctuation_count", "hashtag_count", "mention_count",
    "has_question", "exclamation_count", "presence_keyword", "presence_location"
]

MODELS = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(),
    "SVM": SVC(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results = []

# Boucles principales
for text_name, text_col in TEXT_COLUMNS.items():
    for use_meta in [False, True]:
        selected_features = [text_col] + (META_FEATURES if use_meta else [])
        missing = [col for col in selected_features if col not in df.columns]
        if missing:
            continue

        X = df[selected_features]
        y = df["target"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

        for model_name, model in MODELS.items():
            transformers = [("tfidf", TfidfVectorizer(max_features=3000, ngram_range=(1,2)), text_col)]
            if use_meta:
                transformers.append(("num", StandardScaler(), META_FEATURES))

            preprocessor = ColumnTransformer(transformers)

            pipe = Pipeline([
                ("preprocessing", preprocessor),
                ("clf", model)
            ])

            pipe.fit(X_train, y_train)
            y_pred = pipe.predict(X_test)

            accuracy = accuracy_score(y_test, y_pred)
            precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None, labels=[0, 1])
            results.append({
                "text_column": text_name,
                "use_meta": use_meta,
                "model": model_name,
                "recall_1": recall[1],
                "f1_1": f1[1],
                "accuracy": accuracy
            })

# Résumé final
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="recall_1", ascending=False)
display(df_results[["text_column", "use_meta", "model", "recall_1", "f1_1", "accuracy"]])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:50:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:50:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:50:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [15:50:07] WARN

,text_column,use_meta,model,recall_1,f1_1,accuracy
16,combined_lemma,False,LogisticRegression,0.776765,0.780320,0.781570
8,lemma,False,LogisticRegression,0.772210,0.777523,0.779295
32,combined_stemmed,False,LogisticRegression,0.751708,0.765661,0.770193
24,stemmed,False,LogisticRegression,0.749431,0.758939,0.762230
10,lemma,False,SVM,0.749431,0.766007,0.771331
4,clean_text,True,LogisticRegression,0.747153,0.773585,0.781570
12,lemma,True,LogisticRegression,0.744875,0.770318,0.778157
18,combined_lemma,False,SVM,0.744875,0.765808,0.772469
20,combined_lemma,True,LogisticRegression,0.740319,0.763807,0.771331
34,combined_stemmed,False,SVM,0.740319,0.760234,0.766780


In [ ]:
pd.set_option("display.max_rows", None)
display(df_results[["text_column", "use_meta", "model", "recall_1", "f1_1", "accuracy"]])


,text_column,use_meta,model,recall_1,f1_1,accuracy
56,combined_lemma,False,LogisticRegression,0.776765,0.780320,0.781570
16,combined_lemma,False,LogisticRegression,0.776765,0.780320,0.781570
48,lemma,False,LogisticRegression,0.772210,0.777523,0.779295
8,lemma,False,LogisticRegression,0.772210,0.777523,0.779295
32,combined_stemmed,False,LogisticRegression,0.751708,0.765661,0.770193
72,combined_stemmed,False,LogisticRegression,0.751708,0.765661,0.770193
10,lemma,False,SVM,0.749431,0.766007,0.771331
64,stemmed,False,LogisticRegression,0.749431,0.758939,0.762230
50,lemma,False,SVM,0.749431,0.766007,0.771331
24,stemmed,False,LogisticRegression,0.749431,0.758939,0.762230


In [ ]:
def get_strategy(model_name):
    if model_name == "RandomForest":
        return "Bagging"
    elif model_name == "XGBoost":
        return "Boosting"
    else:
        return "None"

df_results["strategy"] = df_results["model"].apply(get_strategy)


In [ ]:
df_results.groupby("strategy")[["recall_1", "f1_1", "accuracy"]].mean().sort_values(by="recall_1", ascending=False)


,recall_1,f1_1,accuracy
strategy,,,
None,0.719248,0.743539,0.752730
Bagging,0.677904,0.727583,0.747042
Boosting,0.676310,0.727111,0.746530


In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# Charger le fichier CSV
df = pd.read_csv("df_balanced.csv")
y = df["target"]

# Colonnes texte à tester
TEXT_COLUMNS = {
    "clean_text": "clean_text", 
    "lemma": "text_lemma",                          
    "combined_lemma": "combined_lemma",
    "stemmed": "text_stem",                       
    "combined_stemmed": "combined_stemmed"
}

# Méta-features
META_FEATURES = [
    "word_count", "stopword_count", "url_count", "char_count",
    "punctuation_count", "hashtag_count", "mention_count",
    "has_question", "exclamation_count", "presence_keyword", "presence_location"
]

# Modèles et grilles de paramètres
MODELS_PARAMS = {
    "LogisticRegression": (
        LogisticRegression(max_iter=1000),
        {"clf__C": [0.1, 1, 10]}
    ),
    "RandomForest": (
        RandomForestClassifier(),
        {"clf__n_estimators": [50, 100], "clf__max_depth": [None, 10]}
    ),
    "SVM": (
        SVC(),
        {"clf__C": [0.1, 1, 10]}
    ),
    "XGBoost": (
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        {"clf__n_estimators": [50, 100], "clf__max_depth": [3, 6]}
    )
}

results = []

# Boucle sur les colonnes texte et sur les méta-features
for text_name, text_col in TEXT_COLUMNS.items():
    for use_meta in [False, True]:
        selected_features = [text_col] + (META_FEATURES if use_meta else [])
        if not all(col in df.columns for col in selected_features):
            continue

        X = df[selected_features]
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

        for model_name, (model, param_grid) in MODELS_PARAMS.items():
            print(f"🔍 GridSearch - Text: {text_name} | Meta: {use_meta} | Model: {model_name}")

            transformers = [("tfidf", TfidfVectorizer(max_features=3000, ngram_range=(1,2)), text_col)]
            if use_meta:
                transformers.append(("num", StandardScaler(), META_FEATURES))

            preprocessor = ColumnTransformer(transformers)
            pipe = Pipeline([
                ("preprocessing", preprocessor),
                ("clf", model)
            ])

            grid = GridSearchCV(pipe, param_grid, cv=3, scoring='f1', n_jobs=-1)
            grid.fit(X_train, y_train)
            y_pred = grid.predict(X_test)

            score = classification_report(y_test, y_pred, output_dict=True)
            results.append({
                "text_column": text_name,
                "use_meta": use_meta,
                "model": model_name,
                "best_params": grid.best_params_,
                "accuracy": score["accuracy"],
                "recall_1": score["1"]["recall"],
                "f1_1": score["1"]["f1-score"]
            })

# Résumé
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="f1_1", ascending=False)
df_results.to_csv("gridsearch_results.csv", index=False)
print("\n✅ Résultats sauvegardés dans gridsearch_results.csv")


🔍 GridSearch - Text: clean_text | Meta: False | Model: LogisticRegression
🔍 GridSearch - Text: clean_text | Meta: False | Model: RandomForest
🔍 GridSearch - Text: clean_text | Meta: False | Model: SVM
🔍 GridSearch - Text: clean_text | Meta: False | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:02] WARN

🔍 GridSearch - Text: clean_text | Meta: True | Model: LogisticRegression
🔍 GridSearch - Text: clean_text | Meta: True | Model: RandomForest
🔍 GridSearch - Text: clean_text | Meta: True | Model: SVM
🔍 GridSearch - Text: clean_text | Meta: True | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:08] WARN

🔍 GridSearch - Text: lemma | Meta: False | Model: LogisticRegression
🔍 GridSearch - Text: lemma | Meta: False | Model: RandomForest
🔍 GridSearch - Text: lemma | Meta: False | Model: SVM
🔍 GridSearch - Text: lemma | Meta: False | Model: XGBoost


...

🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: LogisticRegression
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: RandomForest
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: SVM
🔍 GridSearch - Text: combined_stemmed | Meta: False | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:45] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:45] WARN

🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: LogisticRegression
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: RandomForest
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: SVM
🔍 GridSearch - Text: combined_stemmed | Meta: True | Model: XGBoost


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [02:40:51] WARN


✅ Résultats sauvegardés dans gridsearch_results.csv


In [ ]:


df_results = pd.read_csv("gridsearch_results.csv")


In [ ]:
# Affiche les top modèles triés par f1-score sur les catastrophes
top_results = df_results.sort_values(by="f1_1", ascending=False)

# Affiche les 5 meilleurs
print(top_results[["text_column", "use_meta", "model", "best_params", "f1_1"]].head(5))


      text_column  use_meta               model      best_params      f1_1
0  combined_lemma     False  LogisticRegression  {'clf__C': 0.1}  0.782007
1           lemma     False  LogisticRegression    {'clf__C': 1}  0.777523
2      clean_text      True  LogisticRegression    {'clf__C': 1}  0.773585
3           lemma      True  LogisticRegression    {'clf__C': 1}  0.770318
4         stemmed      True  LogisticRegression    {'clf__C': 1}  0.769231
